## Importação das LIBS

In [1]:
import pyspark
import os
from delta import *
from pyspark.sql import SparkSession
from pyspark.sql.utils import AnalysisException
import pandas as pd
from configs import config_env_test
from functions import functions as func
import logging


builder = pyspark.sql.SparkSession.builder.appName("LocalDelta") \
    .master("local") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = configure_spark_with_delta_pip(builder).getOrCreate()
print(spark.version)


3.4.1


## Mapping

In [2]:
prefix_layer_name_source = config_env_test.prefix_layer_name['2'] #silver layer
hdfs_source = config_env_test.hdfs_path['silver']

prefix_layer_name_target = config_env_test.prefix_layer_name['3'] #gold layer
hdfs_target = config_env_test.hdfs_path['gold']

## Aggregated gold tables

In [3]:
# Configure the logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Log the start of ingestions
logging.info("Starting Gold aggregated...")

for key, value in config_env_test.tables_postgres_adventureworks.items():
    table = value
    table_name = func.convert_table_name(table)    
    
    try:
        df = spark.read.format("delta").load(f'{hdfs_source}{prefix_layer_name_source}{table_name}')  
        df.write.format("delta").mode("overwrite").save(f'{hdfs_target}{prefix_layer_name_target}{table_name}')        
        logging.info(f"Table {table_name} successfully processed and saved to HDFS: {hdfs_target}{prefix_layer_name}{table_name}")
        
    except Exception as e:
        logging.error(f"Error processing table {table}: {str(e)}")

logging.info("Gold aggregated completed!")

2024-01-16 23:49:53,879 - INFO - Starting Gold aggregated...


+-----------------+------------+--------------------+
|countryregioncode|currencycode|        modifieddate|
+-----------------+------------+--------------------+
|               AE|         AED|2014-02-08 10:17:...|
|               AR|         ARS|2014-02-08 10:17:...|
|               AT|         ATS|2014-02-08 10:17:...|
|               AT|         EUR| 2008-04-30 00:00:00|
|               AU|         AUD|2014-02-08 10:17:...|
|               BB|         BBD|2014-02-08 10:17:...|
|               BD|         BDT|2014-02-08 10:17:...|
|               BE|         BEF|2014-02-08 10:17:...|
|               BE|         EUR| 2008-04-30 00:00:00|
|               BG|         BGN|2014-02-08 10:17:...|
|               BH|         BHD|2014-02-08 10:17:...|
|               BN|         BND|2014-02-08 10:17:...|
|               BO|         BOB|2014-02-08 10:17:...|
|               BR|         BRL|2014-02-08 10:17:...|
|               BS|         BSD|2014-02-08 10:17:...|
|               BT|         

2024-01-16 23:50:14,030 - ERROR - Error processing table sales.countryregioncurrency: name 'prefix_layer_name' is not defined


+------------+-------------+--------------+--------+-------+-------------------+
|creditcardid|     cardtype|    cardnumber|expmonth|expyear|       modifieddate|
+------------+-------------+--------------+--------+-------+-------------------+
|           1| SuperiorCard|33332664695310|      11|   2006|2013-07-29 00:00:00|
|           2|  Distinguish|55552127249722|       8|   2005|2013-12-05 00:00:00|
|           3|ColonialVoice|77778344838353|       7|   2005|2014-01-14 00:00:00|
|           4|ColonialVoice|77774915718248|       7|   2006|2013-05-20 00:00:00|
|           5|        Vista|11114404600042|       4|   2005|2013-02-01 00:00:00|
|           6|  Distinguish|55557132036181|       9|   2006|2014-04-10 00:00:00|
|           7|  Distinguish|55553635401028|       6|   2007|2013-02-01 00:00:00|
|           8| SuperiorCard|33336081193101|       7|   2007|2013-06-30 00:00:00|
|           9|  Distinguish|55553465625901|       2|   2005|2013-09-23 00:00:00|
|          10| SuperiorCard|

2024-01-16 23:50:23,049 - ERROR - Error processing table sales.creditcard: name 'prefix_layer_name' is not defined


...

2024-01-16 23:52:21,269 - ERROR - Error processing table sales.specialoffer: name 'prefix_layer_name' is not defined


+--------------+---------+--------------------+-------------------+
|specialofferid|productid|             rowguid|       modifieddate|
+--------------+---------+--------------------+-------------------+
|             1|      680|bb30b868-d86c-455...|2011-04-01 00:00:00|
|             1|      706|b3c9a4b1-2ae6-4cb...|2011-04-01 00:00:00|
|             1|      707|27b711fe-0b77-4ea...|2011-04-01 00:00:00|
|             1|      708|46cbb78b-246e-4d6...|2011-04-01 00:00:00|
|             1|      709|cf102aa0-055f-4d2...|2011-04-01 00:00:00|
|             1|      710|63718da1-464b-432...|2011-04-01 00:00:00|
|             1|      711|457eb971-d1c9-48c...|2011-04-01 00:00:00|
|             1|      712|5b948448-bae5-4f2...|2011-04-01 00:00:00|
|             1|      713|07768f40-6e46-430...|2011-04-01 00:00:00|
|             1|      714|85004bce-c74a-4d4...|2011-04-01 00:00:00|
|             1|      715|be469616-b279-452...|2011-04-01 00:00:00|
|             1|      716|96d141fd-1d8e-431...|2

2024-01-16 23:52:29,935 - ERROR - Error processing table sales.specialofferproduct: name 'prefix_layer_name' is not defined


+----------------+--------------------+-------------+--------------------+--------------------+--------------------+
|businessentityid|                name|salespersonid|        demographics|             rowguid|        modifieddate|
+----------------+--------------------+-------------+--------------------+--------------------+--------------------+
|             292|Next-Door Bike Store|          279|<StoreSurvey xmln...|a22517e3-848d-4eb...|2014-09-12 11:15:...|
|             294|Professional Sale...|          276|<StoreSurvey xmln...|b50ca50b-c601-4a1...|2014-09-12 11:15:...|
|             296|      Riders Company|          277|<StoreSurvey xmln...|337c3688-1339-4e1...|2014-09-12 11:15:...|
|             298|  The Bike Mechanics|          275|<StoreSurvey xmln...|7894f278-f0c8-4d1...|2014-09-12 11:15:...|
|             300|   Nationwide Supply|          286|<StoreSurvey xmln...|c3fc9705-a8c4-4f3...|2014-09-12 11:15:...|
|             302|Area Bike Accesso...|          281|<StoreSurve

2024-01-16 23:52:37,803 - ERROR - Error processing table sales.store: name 'prefix_layer_name' is not defined
2024-01-16 23:52:37,804 - INFO - Gold aggregated completed!
